Import Libraries

In [1]:
import pandas as pd
import numpy as np
import Saba_Scrapping as SS
from geopy.geocoders import Nominatim
import routing_osrm as Rosrm

Scrap Data --> Infosaba.com

In [3]:
my_chromdriver = "Z:\\HQ\\BDM\\a.zohdi\\Data Engineering\\Github\\Geocoding\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

df_industries = SS.GET_industries(my_chromdriver)
print('Industries are Done !')

urls = df_industries['URL']
df_units = SS.GET_unit_subjects(urls, my_chromdriver)
print('Units are Done !')

df_complexes = SS.GET_complexes(my_chromdriver)
print('Complexes are Done !')

urls = df_units['URL']
df_unitcomplex = SS.GET_unit_complex(urls, my_chromdriver)
print('Unit_Complex is Done !')

Industries are Done !
Units are Done !
Complexes are Done !
There is no Complex in this link: https://infosaba.com/Unit_Subjects/بازیافت-پروپیلن
There is no Complex in this link: https://infosaba.com/Unit_Subjects/بوتانول
There is no Complex in this link: https://infosaba.com/Unit_Subjects/Staple
There is no Complex in this link: https://infosaba.com/Unit_Subjects/اپکسی-رزین
There is no Complex in this link: https://infosaba.com/Unit_Subjects/تاسیسات-استخراج-سنگ-آهن
There is no Complex in this link: https://infosaba.com/Unit_Subjects/ورق-خودرو
There is no Complex in this link: https://infosaba.com/Unit_Subjects/مشخصات-معادن-بوکسیت
There is no Complex in this link: https://infosaba.com/Unit_Subjects/کوره‌-شمش-ریزی-آلومینیوم
There is no Complex in this link: https://infosaba.com/Unit_Subjects/اکسیداسیون
There is no Complex in this link: https://infosaba.com/Unit_Subjects/بروشوئی-سیائیدی
There is no Complex in this link: https://infosaba.com/Unit_Subjects/خردایش
There is no Complex in thi

Get Coordinations using complex.name & OSM.name (kinda)

In [2]:
complex_coords = pd.read_excel("Z:\\HQ\\BDM\\a.zohdi\\Complex OSM.xlsx", index_col=0)
geolocator = Nominatim(user_agent="Zohdi")

# getting coordinations of complexes
complex_coords['coordination'] = np.nan
i = 0
while i < len(complex_coords):
    if not pd.isna(complex_coords.iloc[i, 3]):

        OSM_name = complex_coords.iloc[i, 3]
        location = geolocator.geocode(OSM_name)
        coordination = f"{location.latitude}, {location.longitude}"
        complex_coords.iloc[i, 5] = coordination
        i += 1

    else:

        complex_coords.iloc[i, 5] = complex_coords.iloc[i, 4]
        i += 1

# split coordination to lat-long
complex_coords[['latitude', 'longitude']] = complex_coords['coordination'].str.split(',', expand=True)

In [9]:
df_complexes = complex_coords

with pd.ExcelWriter('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx') as writer:
    df_industries.to_excel(writer, sheet_name='Industry')
    df_units.to_excel(writer, sheet_name='Unit')
    df_complexes.to_excel(writer, sheet_name='Complex')
    df_unitcomplex.to_excel(writer, sheet_name='Unit_Complex')

In [2]:
df_industries = pd.read_excel('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx', sheet_name='Industry', index_col=0)
df_units = pd.read_excel('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx', sheet_name='Unit', index_col=0)
df_complexes = pd.read_excel('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx', sheet_name='Complex', index_col=0)
df_unitcomplex = pd.read_excel('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx', sheet_name='Unit_Complex', index_col=0)

Slicing Unit_Complex Dataframe for each Unit

In [3]:
# Transform capacity_value & under_construction_value 

df_unitcomplex['capacity_value'] = df_unitcomplex['capacity_value'].str.replace(",","")
df_unitcomplex['under_construction_value'] = df_unitcomplex['under_construction_value'].str.replace(",","")

# Transform coordination

df_complexes['coordination'] = df_complexes['coordination'].str.split(", ")

In [4]:
df_unitcomplex_joined = df_unitcomplex.merge(df_complexes, left_on='complex.name', right_on='name')
df_unitcomplex_joined.drop(columns=['unit.url', 'name', 'city', 'industry.name', 'OSM_name', 'GoogleMap_coordination'], inplace=True)

for i in df_units[df_units['industry.id'] == 4].index:
    exec("df_unitcomplex_"+str(i)+" = df_unitcomplex_joined[df_unitcomplex_joined['unit.id'] == i]")
    exec("df_unitcomplex_"+str(i)+".dropna(subset=['coordination'], inplace=True)")

<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:1: SettingWithCopyW

In [7]:
result = Rosrm.table(sources=list(df_unitcomplex_128['coordination']),
                     sources_reference=list(df_unitcomplex_128['complex.name']),
                     destinations=list(df_unitcomplex_130['coordination']),
                     destinations_reference=list(df_unitcomplex_130['complex.name']))

In [10]:
result['distance_km']

,مجتمع معدنی و صنعتی گل گهر,مجتمع فولاد چادرملو,مجتمع سنگ آهن گهر زمین,مجتمع صنعتی و معدنی اپال پارسیان سنگان,مجتمع فولاد سنگان خراسان,مجتمع فولاد زرند ایرانیان (ذوب آهن زرند کرمان),مجتمع فولاد سیرجان ایرانیان,مجتمع صنعتی ذوب آهن پاسارگاد,مجتمع سنگ آهن مرکزی ایران,مجتمع سنگ آهن سنگان,...,کارخانه تولید کنسانتره سنگ آهن هماتیت رضوان,مجتمع آندیا فولاد زنجان,مجتمع ذوب آهن غرب کشور,مجتمع کنسانتره سنگ آهن فکور صنعت,مجتمع آریا جنوب ایرانیان (کرمان -زرند),مجتمع صنعتی و معدنی جهان نمو,مجتمع نورد فولاد صنعتی و ساختمانی یزد,مجتمع کیمیا معادن سپاهان,مجتمع سپهر صنعت آرتا,مجتمع سنگ آهن تنگ زاغ
معدن سنگ آهن سنگان,1080.7246,898.8912,1081.6619,1.9162,3.8938,838.6028,908.2709,1353.9438,898.1772,3.3513,...,1292.2261,1444.4898,1403.4335,832.7624,835.9963,1393.5754,860.6391,1546.6998,1443.3801,1213.7862
معدن سنگ آهن گل گهر,2.1600,495.6609,21.7360,1074.6051,1074.5398,307.5730,191.5596,362.9974,374.8493,1076.5720,...,316.6322,1280.9492,1212.2205,352.2325,300.7615,1230.0348,400.9477,1383.1592,1252.1670,238.1924
معدن چادرملو,467.9148,225.5087,468.8521,754.8235,754.7582,249.2742,383.0031,680.5611,101.7246,756.7904,...,681.4626,1010.7970,942.0682,262.1371,254.0785,959.8826,187.2565,1113.0070,982.0148,603.0229
معدن سنگ آهن آنومالی شمالی,381.4781,231.7626,382.4154,902.1317,902.0664,197.9250,322.8338,619.5503,16.9721,904.0986,...,595.0260,1017.0509,948.3221,175.7004,167.6418,966.1365,137.0494,1119.2609,988.2686,516.5862
معدن سنگ آهن سه چاهون,407.2514,259.9247,408.1887,854.4013,854.3360,212.8470,337.7557,647.7125,41.0612,856.3682,...,620.7993,1045.2130,976.4842,201.4737,193.4151,994.2986,165.2115,1147.4230,1016.4308,542.3595
معدن سنگ آهن جلال آباد,351.1325,365.9183,352.0697,830.5346,830.4693,53.7699,178.6787,693.3311,165.9657,832.5015,...,562.6340,1151.2066,1082.4778,2.8412,30.7494,1100.2922,271.2051,1253.4166,1122.4244,484.1941
معدن سنگ آهن چغارت,378.9088,231.5821,379.8461,885.8136,885.7482,195.3557,320.2645,619.3699,12.7186,887.7804,...,592.4567,1016.8704,948.1416,173.1311,165.0725,965.9560,136.8689,1119.0805,988.0881,514.0169
معدن سنگ آهن چاه گز,448.0196,300.6930,448.9569,817.6321,817.5668,264.4666,398.7438,688.4807,81.8295,819.5990,...,661.5675,1085.9812,1017.2524,242.2420,234.1833,1035.0668,205.9798,1188.1912,1057.1990,583.1278
معدن سنگ آهن گلالی,1221.7140,742.9200,1222.6512,1414.9918,1414.9265,1117.5625,1191.4392,994.1539,953.2558,1416.9588,...,1435.2620,252.9535,30.3813,1095.3379,1087.2792,309.7340,826.1157,209.8391,53.9668,1356.8222
معدن سنگ آهن شهرک بیجار,1380.9420,902.1479,1381.8792,1510.7676,1510.7022,1276.7905,1350.6671,1224.8385,1112.4836,1512.7345,...,1594.4899,111.0188,230.2621,1254.5658,1246.5072,164.6539,985.3436,4.7465,193.9301,1516.0501


In [6]:
list(df_unitcomplex_128['coordination'])
dd = list(df_unitcomplex_130['coordination'])
dd[0]

['29.0918765', '55.3341122']